In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import seaborn as sns

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
df=dataset.copy()
df.head()

In [ ]:
# Any necessity to impute any value?
df.isna()\
  .sum()


In [ ]:
# One-hot encode categorical values to be parsed as numerical ones
categoricals = ['sex', 'smoker', 'region']
df = dataset.copy() # Start with a fresh copy of the original dataset
df = pd.get_dummies(df, columns = categoricals, drop_first = False)
df.head()

In [ ]:
train_dataset=df.sample(frac=0.8,random_state=42)
test_dataset=df.drop(train_dataset.index)

train_labels=train_dataset.pop('expenses')
test_labels=test_dataset.pop('expenses')

In [ ]:
smoker_counts = df['smoker_yes'].value_counts()
plt.figure(figsize=(5, 5))
plt.pie(smoker_counts, labels=smoker_counts.index, autopct='%1.1f%%', colors=sns.color_palette('pastel'))
plt.title("Smoker Distribution")
plt.show()

In [ ]:
#Add output layers
input_shape=len(train_dataset.keys())
model=keras.Sequential(
    [
    layers.Dense(128,
                 activation = 'relu',
                 input_shape = [input_shape]),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(16, activation = 'relu'),
    layers.Dense(1, activation = 'relu')

    ]
)
optimizer = tf.keras.optimizers.Adam(
    learning_rate = .008
)

model.compile(loss = 'mae',
              optimizer = optimizer,
              metrics = ['mae',
                         'mse'])

model.summary()

In [ ]:
EPOCHS = 500
checkpoint_filepath = './tmp/ckpt/fcchealthcosts.model.keras'

checkpoint_callback  = [tf.keras.callbacks\
                        .ModelCheckpoint(filepath = checkpoint_filepath,
                                         monitor = 'val_loss',
                                         verbose = 1,
                                         save_best_only = True,
                                         mode = 'min'
                                         )
]

history = model.fit(train_dataset,
                    train_labels,
                    epochs = EPOCHS,
                    verbose = 1,
                    validation_split = .2,
                    shuffle = False,
                    callbacks = checkpoint_callback
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
